In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 11.6 MB/s eta 0:00:00


In [ ]:
!pip install cv2
!pip install PIL

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# valid set 생성
from PIL import Image
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import os

transform = A.Compose([
    A.Transpose(p=1)
])

input_folder = "/content/train_img" # 원본 이미지 폴더 경로
output_folder = "/content/val/images" # 변형 이미지 폴더 경로


# input_folder 안의 모든 이미지 파일을 가져와서 변형을 적용하고 저장합니다.
for filename in os.listdir(input_folder):
    if filename.endswith(".png"):
        image = cv2.imread(os.path.join(input_folder, filename))

        transformed = transform(image=image)
        transformed_image = transformed["image"]

        output_path = os.path.join(output_folder, "transpose_" + filename)
        cv2.imwrite(output_path, transformed_image)
def swap_coordinates(line):
    coordinates = line.split()  # 공백을 기준으로 좌표들을 분리
    # x와 y 좌표를 서로 바꾸기
    swapped_coordinates = ' '.join([coordinates[i + 1] if i % 2 == 0 else coordinates[i - 1] for i in range(len(coordinates))])
    return swapped_coordinates

label_input_folder = "/content/label/home/hawk/decs_jupyter_lab/dataset/converted_4" # 원본 이미지 폴더 경로
label_output_folder = "/content/val/labels" # 변형 이미지 폴더 경로

for filename in os.listdir(label_input_folder):
    if filename.endswith(".txt"):
        iinput_file = os.path.join(label_input_folder, filename)
        ooutput_file = os.path.join(label_output_folder, "transpose_" + filename)
        with open(iinput_file, 'r') as input_file:
            lines = input_file.readlines()

        with open(ooutput_file, 'w') as output_file:
            for line in lines:
                swapped_line = swap_coordinates(line[1::])
                output_file.write('0 ' + swapped_line + '\n')

        output_path = os.path.join(label_output_folder, "transpose_" + filename)


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8x-seg.pt")

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8x-seg") # segmentation 모델 : "yolov8x-seg", 모델이 너무 크면 yolov8n,s,m,l 사용가능
results = model.train(
    data="/content/drive/MyDrive/Hoin.yaml",
    imgsz= 1024,
    epochs= 150,
    batch= 8,
    patience=5,
    workers=16,
    exist_ok=True,
    project="Aerial_Segmentation",
    name="train",
    seed=6666666,
    pretrained=False,
    resume=False,
    optimizer="NAdam",
    lr0=1e-3,
    cos_lr=True,
    val=True,
    mask_ratio=4,
    overlap_mask=True
    )

Ultralytics YOLOv8.0.143 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=/content/drive/MyDrive/Hoin.yaml, epochs=150, patience=5, batch=8, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=16, project=Aerial_Segmentation, name=train, exist_ok=True, pretrained=False, optimizer=NAdam, verbose=True, seed=6666666, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=

In [ ]:
mask = result[0].masks
mask_xy_8 = mask.data
print(mask_xy_8.shape)
mask_xy= mask_xy_8.any(dim=0).int()
mask_cpu= mask_xy.cpu()
mask_rle = rle_encode(mask_cpu.numpy())
"""mask_rle = rle_encode(mask_cpu.numpy())
temp = rle_decode(mask_rle,(448,448))
temp2 = np.zeros((224,224))
for i in range(448) :
  for j in range(448) :
    if temp[i][j] == 1:
      i_reduce = int(i / 2)
      j_reduce = int(j / 2)
      temp2[i_reduce][j_reduce] = 1
temp3 = rle_encode(temp2)
print(temp3)"""


# mask_224를 numpy 배열로 변환합니다.

# 다운샘플링된 temp2를 이용하여 새로운 RLE 인코딩을 생성합니다.


torch.Size([2, 224, 224])


'mask_rle = rle_encode(mask_cpu.numpy())\ntemp = rle_decode(mask_rle,(448,448))\ntemp2 = np.zeros((224,224))\nfor i in range(448) :\n  for j in range(448) :\n    if temp[i][j] == 1:\n      i_reduce = int(i / 2)\n      j_reduce = int(j / 2)\n      temp2[i_reduce][j_reduce] = 1\ntemp3 = rle_encode(temp2)\nprint(temp3)'

In [ ]:
results = model.predict("/content/test_img",
                          imgsz=(224,224), iou=0.7, conf=0.25,
                          save_conf=False, save=True, save_txt=False,
                          name="predict", exist_ok=True, device=0,verbose=False, show_labels=False, retina_masks=True, augment=False)

Results saved to runs/segment/predict
